In [1]:
from Signal_generator.generate_signal import generate_X_matrix
from Algorithmes.beamforming import beamforming_method
from Signal_generator.generate_signal import generate_A_matrix
from Signal_generator.generate_signal import generate_S_matrix
from Signal_generator.generate_signal import generate_noise
from Signal_generator.generate_signal import generate_R_hat
from Signal_generator.generate_signal import generate_R_hat_with_phase
from Algorithmes.music import music_method
from Algorithmes.music import estimate_angles
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import itertools
from torchsummary import summary
from numpy.linalg import eigh
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
import math
import random
from sklearn.metrics import mean_squared_error
from Algorithmes.music import generate_steering_vector
from Plots.draw_plot import plot_single_music

In [2]:
nbSources = 1 # Nombre de sources  dans l'article ils fixent le nombre de sources à 2
nbSensors = 9 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels


#Pour l'échantillon d'entraînements ils utilisent une observation de chaque combinaison d'angles d'arrivée possible pour chaque SNR level, dans le cas d'une seule source, 2G+1 observations
#SNR ratio dans le papier est seulement low, de -20 à 0 DB avec un pas de 5
#Petite note : si on fait un modèle pour chaque SNR level, les performances augmentent légérements, mais cela ne vaut pas l'effort effectué
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
L = nbSensors
T = 400
correlation_List = [] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.
SNR_TRAIN = 30
Q = 8
N = 2**12 
theta_start = -45
theta_final = 45


In [3]:
import numpy as np

def calculate_music_spectrum(R_hat, nbSensors, nbSources, theta_range):
    eigenvalues, eigenvectors = np.linalg.eigh(R_hat)
    noise_subspace = eigenvectors[:, :nbSensors - nbSources]
    music_spectrum = np.zeros(len(theta_range))

    for i, theta in enumerate(theta_range):
        steering_vector = generate_steering_vector(nbSensors, theta)
        music_spectrum[i] = 1 / np.abs(np.conj(steering_vector).T @ noise_subspace @ noise_subspace.conj().T @ steering_vector)

    return music_spectrum



In [4]:
def generate_deepmusic_partitioned_data(nbSensors, nbSources, T, J_alpha, J_beta, SNR_TRAIN, Q, N, theta_start, theta_final):
    training_data_sets = [[] for _ in range(Q)]

    for j_alpha in range(J_alpha):
        thetaList = np.random.uniform(theta_start, theta_final, nbSources).tolist()

        for j_beta in range(J_beta):
            varList = np.ones(nbSources).tolist()
            correlation_List = np.zeros(nbSources * (nbSources - 1) // 2).tolist()
            X = generate_X_matrix(nbSources, nbSensors, T, thetaList, varList, correlation_List, SNR_TRAIN)
            
            R_hat = generate_R_hat(X)
            full_theta = np.linspace(theta_start, theta_final, N)
            full_music_spectrum = calculate_music_spectrum(R_hat, nbSensors, nbSources, full_theta)

            for q in range(Q):
                # Sampling MUSIC spectrum for the q-th subregion
                start_index = q * L
                end_index = start_index + L
                pq = full_music_spectrum[start_index:end_index]
                
                input_data = generate_R_hat_with_phase(X)
                output_data = pq
                training_data_sets[q].append((input_data, output_data))                

    return training_data_sets

# Example usage
J_alpha = 100  # Number of DOA angle sets to generate
J_beta = 10   # Number of iterations for each set of DOA angles
SNR_TRAIN_list = [15,20, 25, 30] # Different SNR levels for training
theta_range = np.linspace(theta_start, theta_final, N*Q)
training_datasets = generate_deepmusic_partitioned_data(nbSensors, nbSources, T, J_alpha, J_beta, SNR_TRAIN, Q, N,theta_start, theta_final)


[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class DeepMusicDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_data, output_data = self.data[idx]
        return torch.tensor(input_data, dtype=torch.float32), torch.tensor(output_data, dtype=torch.float32)

def create_loaders_for_all_subregions(Q, training_datasets, batch_size=128, validation_split=0.2, test_split=0.2):
    train_loaders = []
    val_loaders = []
    test_loaders = []

    for q in range(Q):
        train_data, test_data = train_test_split(training_datasets[q], test_size=test_split, random_state=42)
        train_data, val_data = train_test_split(train_data, test_size=validation_split, random_state=42)

        train_dataset = DeepMusicDataset(train_data)
        val_dataset = DeepMusicDataset(val_data)
        test_dataset = DeepMusicDataset(test_data)

        train_loaders.append(DataLoader(train_dataset, batch_size=batch_size, shuffle=True))
        val_loaders.append(DataLoader(val_dataset, batch_size=batch_size, shuffle=False))
        test_loaders.append(DataLoader(test_dataset, batch_size=batch_size, shuffle=False))

    return train_loaders, val_loaders, test_loaders

# Utilisation de la fonction
train_loaders, val_loaders, test_loaders = create_loaders_for_all_subregions(Q, training_datasets)

In [25]:

class DeepMusicModel(nn.Module):
    def __init__(self):
        super(DeepMusicModel, self).__init__()
        # Define the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(num_features=256)
        
        # Define the second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(num_features=256)
        
        # Define the third convolutional layer
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=256)
        
        # Define the fourth convolutional layer
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=256)
        
        # Assuming the spatial dimensions (height and width) are reduced to 1x1 after the convolutions
        # Define the fully connected layers
        self.fc1 = nn.Linear(in_features=20736, out_features=9)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # Softmax layer 
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Apply the first convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn1(self.conv1(x)))
        
        # Apply the second convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn2(self.conv2(x)))
        
        # Apply the third convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn3(self.conv3(x)))
        
        # Apply the fourth convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn4(self.conv4(x)))
        
        # Flatten the tensor for the fully connected layer
        x = torch.flatten(x, 1)
        
        # Apply the first fully connected layer
        x = self.fc1(x)
        
        # Apply the dropout layer
        x = self.dropout(x)

        #Apply the softmax layer
        x = self.softmax(x)
        
        return x

class MultipleDeepMusicModels(nn.Module):
    def __init__(self, q):
        super(MultipleDeepMusicModels, self).__init__()
        self.models = nn.ModuleList([DeepMusicModel() for _ in range(q)])

    def forward(self, x):
        # Générer les sorties pour chaque modèle et chaque élément de x
        outputs = [model(x_elem) for model, x_elem in zip(self.models, x)]

        # Concaténer toutes les sorties en une seule
        concatenated_outputs = torch.cat(outputs, dim=1)  # Ajustez la dimension selon la structure de vos données
        return concatenated_outputs

In [26]:

class DeepMusicModel(nn.Module):
    def __init__(self):
        super(DeepMusicModel, self).__init__()
        # Define the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(num_features=256)
        
        # Define the second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(num_features=256)
        
        # Define the third convolutional layer
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=256)
        
        # Define the fourth convolutional layer
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=256)
        
        # Assuming the spatial dimensions (height and width) are reduced to 1x1 after the convolutions
        # Define the fully connected layers
        self.fc1 = nn.Linear(in_features=20736, out_features=9)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # Softmax layer 
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Apply the first convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn1(self.conv1(x)))
        
        # Apply the second convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn2(self.conv2(x)))
        
        # Apply the third convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn3(self.conv3(x)))
        
        # Apply the fourth convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn4(self.conv4(x)))
        
        # Flatten the tensor for the fully connected layer
        x = torch.flatten(x, 1)
        
        # Apply the first fully connected layer
        x = self.fc1(x)
        
        # Apply the dropout layer
        x = self.dropout(x)

        #Apply the softmax layer
        x = self.softmax(x)
        
        return x

class MultipleDeepMusicModels(nn.Module):
    def __init__(self, q):
        super(MultipleDeepMusicModels, self).__init__()
        self.models = nn.ModuleList([DeepMusicModel() for _ in range(q)])

    def forward(self, x):
        outputs = [model(x_elem) for model, x_elem in zip(self.models, x)]
        return outputs

model = MultipleDeepMusicModels(Q)


In [27]:
#Définition du device

device = (
    "gpu"
    if torch.cuda.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [28]:
import torch
import torch.nn.functional as F
import numpy as np

def train_and_evaluate_spectrum(train_loaders, val_loaders, test_loaders, epochs, nbSources, angles_range, peak_tolerance=5):
    Q = len(train_loaders)
    multi_model = MultipleDeepMusicModels(Q)
    optimizer = torch.optim.SGD(multi_model.parameters(), lr=0.01, momentum=0.9)
    criterion = torch.nn.MSELoss()

    best_val_accuracy = 0
    no_improve_epoch = 0

    for epoch in range(epochs):
        # Training loop
        multi_model.train()
        for batch_per_subregion in zip(*train_loaders):
            inputs = torch.cat([data[0] for data in batch_per_subregion], dim=1)
            targets = torch.cat([data[1] for data in batch_per_subregion], dim=1)
            optimizer.zero_grad()
            outputs = multi_model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        # Validation loop
        multi_model.eval()
        val_loss = 0
        val_accuracies = []
        with torch.no_grad():
            for batch_per_subregion in zip(*val_loaders):
                inputs = torch.cat([data[0] for data in batch_per_subregion], dim=1)
                targets = torch.cat([data[1] for data in batch_per_subregion], dim=1)
                outputs = multi_model(inputs)
                val_loss += criterion(outputs, targets).item()
                val_accuracies.append(calculate_accuracy(outputs, targets, angles_range, nbSources, peak_tolerance))

        avg_val_accuracy = np.mean(val_accuracies)
        if avg_val_accuracy > best_val_accuracy:
            best_val_accuracy = avg_val_accuracy
            no_improve_epoch = 0
        else:
            no_improve_epoch += 1
            if no_improve_epoch >= 3:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # Learning rate decay
        if epoch % 10 == 9:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= 0.5

        print(f'Epoch {epoch}: Avg Val Loss {val_loss / len(val_loaders[0])}, Avg Val Accuracy {avg_val_accuracy}')

    # Test evaluation
    total_mse = 0
    total_angle_accuracy = 0
    test_batches = 0
    multi_model.eval()
    with torch.no_grad():
        for inputs_per_subregion in zip(*test_loaders):
            inputs = [data[0] for data in inputs_per_subregion]
            targets = torch.cat([data[1] for data in inputs_per_subregion], dim=1)
            outputs = multi_model(inputs)
            total_mse += F.mse_loss(outputs, targets).item()
            total_angle_accuracy += calculate_accuracy(outputs, targets, angles_range, nbSources, peak_tolerance)
            test_batches += 1

    avg_test_mse = total_mse / test_batches
    avg_test_angle_accuracy = total_angle_accuracy / test_batches
    print(f'Average MSE on Test Set: {avg_test_mse:.4f}')
    print(f'Average Angle Accuracy on Test Set: {avg_test_angle_accuracy:.4f}')



theta_range = np.linspace(theta_start, theta_final, N*Q)
# Example usage
train_and_evaluate_spectrum(train_loaders, val_loaders, test_loaders, 5, nbSources, theta_range, 5)


RuntimeError: Given groups=1, weight of size [256, 3, 5, 5], expected input[1, 24, 9, 9] to have 3 channels, but got 24 channels instead